In [45]:
import numpy as np

## Input Parameters

In [46]:
# geometry
L = 1.0
w = 0.05
h = 0.1

# mesh
shape = (100, 10)

# material
E = 1000
nu = 0.2

# load
F = 100

## Analytic Bernoulli Solution

In [47]:
EI = E * w * h**3 / 12
UY_Bernoulli = -F * L**3 / (3 * EI)

## Finite Element Solution

In [48]:
# Hooke model
A = np.array([
    [1, nu, 0], 
    [nu, 1, 0],
    [0, 0, (1-nu)/2]]) * (E /(1-nu**2)) * w

In [49]:
from sigmaepsilon.mesh import grid, Q9, PolyData
coords, topo = grid(size=(L, h), shape=shape, eshape='Q9')
coords_ = np.zeros((coords.shape[0], 3))
coords_[:, :2] = coords
coords = coords_
mesh = PolyData(coords=coords, topo=topo, celltype=Q9)

In [50]:
# essential boundary conditions
cond_ebc = np.abs(coords[:, 0]) <= 1e-5
ebcinds = np.where(cond_ebc)[0]
fixity = np.zeros((coords.shape[0], 2), dtype=bool)
fixity[ebcinds, :] = True

In [51]:
# natural boundary conditions
target = np.array([L, h/2, 0])
nbcinds = [mesh.index_of_closest_point(target)]
loads = np.zeros((coords.shape[0], 2))
loads[nbcinds, 1] = -F

In [52]:
from dewloosh.solid.fem import FemMesh, Structure, Q9M
mesh = FemMesh(coords=coords, topo=topo, celltype=Q9M, 
               fixity=fixity, loads=loads, model=A)
structure = Structure(mesh=mesh)

In [53]:
structure.linsolve()
structure.summary

{'preproc': {'sparsify': False},
 'proc': {'time [ms]': 43.00045967102051,
  'N': 8442,
  'use_umfpack': True,
  'permc_spec': 'COLAMD',
  'solver': 'pardiso'},
 'postproc': {},
 'number of elements': 1000,
 'number of nodes': 4221,
 'dofs per node': 2}

In [54]:
UY_FEM = structure.pointdata.dofsol.to_numpy()[nbcinds[0]][1]

## Comparison

In [55]:
print("Analytic Bernoulli Solution : {}".format(UY_Bernoulli))
print("FEM Solution : {}".format(UY_FEM))
diff = 100 * (UY_FEM - UY_Bernoulli) / UY_Bernoulli
print("Difference : {} %".format(diff))
structure.plot()

Analytic Bernoulli Solution : -7999.999999999998
FEM Solution : -8054.064608658752
Difference : 0.6758076082344248 %
